# Extract

In [1]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Create filepaths to read
filepath_college_type = 'Resources/salaries-by-college-type.csv'
filepath_salaries = 'Resources/salaries-by-region.csv'

In [5]:
# Read CSV files and store as dataframes
college_type_df = pd.read_csv(filepath_college_type)
salaries_df = pd.read_csv(filepath_salaries)

# Transform

In [10]:
# Start with college_type data, drop unnecessary columns
college_type = college_type_df[['School Name','School Type']].copy()

# Rename columns (lower-case, no spaces)
college_type.rename(columns={'School Name':'school_name','School Type':'school_type'},inplace=True)

# Set school_name as index
college_type.set_index('school_name',inplace=True)

,school_type
school_name,
Massachusetts Institute of Technology (MIT),Engineering
California Institute of Technology (CIT),Engineering
Harvey Mudd College,Engineering
"Polytechnic University of New York, Brooklyn",Engineering
Cooper Union,Engineering


# Load